### Follow these steps to convert your file:

1. Move your file in the "input_file" directory
2. Click on "Run" and select "Run all cells"
3. Type the name of your file in the input mask that appears below the second cell (don't add the file extension .csv)
4. Find your converted file in the output folder

In [1]:
# all of the libraries are pre-installed
# --> no need to pip-install anything
import pandas as pd
import numpy as np
from datetime import datetime
import pathlib
from pathlib import Path


In [2]:
#import input data
#determine path
path = pathlib.Path().resolve()
#Input from the user
filename = input("Type in input file name without file extension: ")
#then we process the input file, make the Path + .txt
input_file = filename + '.csv'
#create input file path based on the file name given by the user
input_file_path = Path(path) / 'input_files' / input_file
print(f"Your file path: {input_file_path}")

Type in input file name without file extension:  Sample Travel data


Your file path: /Users/Daniel/portfolio_projects/carbon-accounting-data-input/transport-data-transform/input_files/Sample Travel data.csv


### import the csv files

In [3]:
# read in the Plan A template for transport
df_template = pd.read_csv("business_travel_template.csv", sep=";")

In [4]:
# read in the data from the customer
df_or = pd.read_csv(input_file_path,
                    parse_dates=['Start date', 'End date'])


# perform a mini exploratory data analysis

In [7]:
# this cell is optional
print(f'csv size: {df_or.shape}')
print(f"columns: {list(df_or.columns)}")
print(f"services: {df_or.groupby(['Service'])['Service'].count()}")

csv size: (55, 13)
columns: ['Booking description', 'Service', 'Departure airport code', 'Arrival airport code', 'Cabin class', 'Drop-off city', 'Pick-up city', 'End date', 'Start date', 'Distance', 'Itinerary Type', 'Departure Train Station', 'Arrival Train Station']
services: Service
FlexiPerk Trips Service     1
Flights                     6
Hotels                      9
Other Service              10
Premium Service            16
Refund for hotel            1
Trains                     12
Name: Service, dtype: int64


In [6]:
# for testing purpose only
# delete an airport code
#df_or.loc[0,"Arrival airport code"] = np.nan
#df_or.head(5)


# convert return trips to two individual trips

In [8]:
# first, identify all return trips based on the key word "return"
returns = df_or.loc[df_or['Itinerary Type'] == 'return'].copy()
# swap the airport codes
airport_codes = ['Departure airport code', 'Arrival airport code']
returns = returns.rename(columns={
    airport_codes[0]: airport_codes[1],
    airport_codes[1]: airport_codes[0]})
# swap train stations
train_stations = ['Departure Train Station', 'Arrival Train Station']
returns = returns.rename(columns={
    train_stations[0]: train_stations[1],
    train_stations[1]: train_stations[0]})
#  make the arrival date to new departure date
travel_dates = ['End date', 'Start date']
returns = returns.rename(columns={
    travel_dates[0]: travel_dates[1],
    travel_dates[1]: travel_dates[0]})
# # stack the two DataFrames returns and the original data frame
df_or = pd.concat([df_or, returns], ignore_index=True, axis=0)

# create a data frame with all flights

In [9]:
# FLIGHTS
# filter by flights
flights_or = df_or.loc[(df_or["Service"] == "Flights")]
# create a new data frame according to the template
flights = pd.DataFrame(
    np.nan, index=range(len(flights_or)), columns=df_template.columns)
# add the booking description
flights.loc[:, "Transport type"] = "Flight"
# add the departure date
flights.loc[:, "Departure date"] = flights_or[
    "Start date"].reset_index(drop=True)
# add the cabin classes
flights.loc[
    :, "Flight classes (Economy / Business / First / Unknown)"] = flights_or[
        "Cabin class"].reset_index(drop=True)
# rename the flight classes to the correct key word
# TODO! key words in client's data for Business and First unknown at this point
flights.loc[
    :, "Flight classes (Economy / Business / First / Unknown)"].replace(
    {"economy": "Economy"}, inplace=True)
# fill empty ones with unknown
flights.loc[
    :, "Flight classes (Economy / Business / First / Unknown)"].fillna(
    "Unknown", inplace=True)
# add airport code for departure
flights.loc[
    :, "Origin Location (Flight / Road / Rail)"] = flights_or[
        "Departure airport code"].reset_index(drop=True)
# add airport code for destination
flights.loc[
    :, "Destination Location (Flight / Road / Rail)"] = flights_or[
        "Arrival airport code"].reset_index(drop=True)
# test if all airport codes are available
assert flights.loc[:, ["Origin Location (Flight / Road / Rail)",
                   "Destination Location (Flight / Road / Rail)"]
               ].notna().all().all(), "Airport codes incomplete."


## create a data frame with all train rides

In [10]:
# filter by rail
rail_or = df_or.loc[(df_or["Service"] == "Trains")]
# create a new data frame according to the template
rail = pd.DataFrame(
    np.nan, index=range(len(rail_or)), columns=df_template.columns)
# add the booking description
rail.loc[:, "Transport type"] = "Rail"
# add the departure date
rail.loc[:, "Departure date"] = rail_or["Start date"].reset_index(drop=True)
# add train station for departure
rail.loc[:, "Origin Location (Flight / Road / Rail)"] = rail_or[
    "Departure Train Station"].reset_index(drop=True)
# add train station for destination
rail.loc[:, "Destination Location (Flight / Road / Rail)"] = rail_or[
    "Arrival Train Station"].reset_index(drop=True)
# test if all train stations are available
assert rail.loc[:, ["Origin Location (Flight / Road / Rail)",
                "Destination Location (Flight / Road / Rail)"]
            ].notna().all().all(), "Train stations codes incomplete"


### concatenate the data frames for flights and rail

In [34]:
# %% stack the data frames for flights and rail
df = pd.concat([flights, rail], ignore_index=True, axis=0)
# test if departure dates are complete
assert df.loc[:, "Departure date"].notna().all(), " Departure dates incomplete!"

### adjust the format to match the template

In [35]:
# sort by date
df.sort_values(by='Departure date', inplace = True)
# set all passenger per trip to one
df['Number of employees'].fillna(int(1), inplace=True)
# adjust date formate
df['Departure date'] = df['Departure date'].dt.strftime('%Y-%m-%d')
# add the explanation row
s = df_template.iloc[0,:]
df = pd.concat([s.to_frame().T, df], ignore_index=True, axis=0)

### save the csv file in the output directory

In [36]:
# %% save csv
# create a unique name containing the current time and the original filename
now = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
output_file = f"{filename}_converted_{now}.csv"#.replace(" ", "_")
output_file_path = Path(path) / 'output_files' / output_file
# save the csv
df.to_csv(output_file_path, index=False)
print(f"csv saved to {output_file}")

csv saved to Sample Travel data_converted_2023_06_05_20_51_52.csv


### 2 dos
#### change to inplace operator
#### lint